In [12]:
import numpy as np

Rx = np.array([[5,-1,-2],[-1,5,-1],[-2,-1,5]])
ryx = np.array([1,5.3,-3.9])

1.2.1 a) $$w_0 = R_x^{-1}*r_{yx}$$

In [13]:
w0 = np.linalg.solve(Rx,ryx)
print(w0)

[ 0.2  1.  -0.5]


1.2.1 b) $$E[xe] = E[x(y-xW)] = E[xy] - E[xx] * W = r_{xy} - R_x * W, W = R_x^{-1} * rxy $$<br>
         $$E[xe] = r_{xy} - R_x * R_x^{-1} * rxy = 0$$

1.2.1 c) using LS, assume ergodicity 
$$ \hat{R_x} = \sum(x[k] * x[k])/N $$
$$ \hat{r_{xy}} = \sum(x[k] * y[k])/N

1.2.2 d) when k --> infinity <br>
$$ w[k] = w_0 = R_x^{-1} * rxy $$

1.2.2 e) $$a < 1/\lambda_{max}$$

In [99]:
lamb = np.linalg.eigvals(Rx)
lamb_max = max(lamb)
a = 1/lamb_max
print('elgvals =', lamb)
print('a_max =', a)

elgvals = [2.26794919 7.         5.73205081]
a_max = 0.1428571428571429


1.2.2 f) GD: $$ w[k+1] = w[k] + 2\alpha (r_{yx} - R_xw[k]) $$

In [98]:
import csv
csv_reader = csv.reader(open("assignment1_data.csv"))

x = []
y = []
data = []


for row in csv_reader:
    x.append(row[0])
    y.append(row[1])
    data.append(row)

data = np.array(data)
x = np.array(x)
y = np.array(y)
print('data_shape =',data.shape)



data_shape = (10000, 2)


In [ ]:
for i in data.shape[0]:
    a